#Linea de espera

$$G/G/1$$

In [16]:
from random import random
from random import expovariate as expo
from random import triangular
import numpy as np
import pandas as pd

def lineaE(n):
	t0 = 0
	s = 0 # total de clientes en la fila
	k = 0 # indice
	datos = np.zeros( (n,9) )
	encabezados = [
'cliente',
'tiempo de interllegada',
'tiempo de llegada',
'fila',
'inicio del servicio',
'tiempo de servicio',
'fin del servicio',
'tiempo en el sistema',
'tiempo promedio en el sistema']
	for i in range(n):
		# numero de cliente
		datos[i][0] = i + 1

		# tiempo entre llegadas
		t = expo(1/15) # distribucion de llegadas
		datos[i][1] = t

		# tiempo de llegada
		if i == 0:
			t0 = t
			datos[i][2] = t0
		else:
			t0 += t
			datos[i][2] = t0

		# fila
		if i == 0:
			datos[i][3] = s
		elif t0 < datos[k][6]:
			s += 1
			datos[i][3] = s
		elif t0 > datos[k][6]:
			w = datos[k:i,6][(datos[k:i,6] < datos[i][2])].size
			s = datos[k:i,6].size - w
			k += w # se actualiza el indice
			datos[i][3] = s

		# inicio del servicio
		if i==0:
			datos[i][4] = t
		else:
			datos[i][4] = max( datos[i-1][6] , t0 )

		# distribucion del tiempo de servicio
		u = expo(1/8)
		datos[i][5] = u

		# fin del servicio
		datos[i][6] = datos[i][4] + u

		# tiempo en el sistema
		datos[i][7] = datos[i][6] - t0

		# tiempo promedio en el sistema
		if i == 0:
			datos[i][8] = datos[i][6]
		else:
			datos[i][8] = datos[0:i,7].mean()
	return pd.DataFrame( np.round(datos,2) , columns = encabezados).astype({'cliente':np.int32,
'tiempo de interllegada':np.float32,
'tiempo de llegada':np.float32,
'fila':np.int32,
'inicio del servicio':np.float32,
'tiempo de servicio':np.float32,
'fin del servicio':np.float32,
'tiempo en el sistema':np.float32,
'tiempo promedio en el sistema':np.float32})

simu = lineaE(60).style.hide( axis = "index" ).format( precision = 2 )
simu

cliente,tiempo de interllegada,tiempo de llegada,fila,inicio del servicio,tiempo de servicio,fin del servicio,tiempo en el sistema,tiempo promedio en el sistema
1,4.69,4.69,0,4.69,18.43,23.12,18.43,23.12
2,4.29,8.98,1,23.12,18.13,41.25,32.27,18.43
3,5.25,14.23,2,41.25,3.93,45.18,30.95,25.35
4,20.04,34.27,2,45.18,1.33,46.51,12.24,27.22
5,37.03,71.30,0,71.30,20.93,92.23,20.93,23.47
6,27.07,98.37,0,98.37,35.47,133.84,35.47,22.96
7,15.55,113.92,1,133.84,15.05,148.89,34.97,25.05
8,0.22,114.14,2,148.89,0.36,149.25,35.11,26.47
9,42.71,156.85,0,156.85,12.57,169.42,12.57,27.55
10,0.43,157.28,1,169.42,4.03,173.45,16.17,25.88


In [18]:
k = 100000
simu2 = lineaE(k)
print("Clientes promedio en la fila:",simu2['fila'].mean())
print("Probabilidad de cero clientes en la fila:", simu2["fila"][(simu2["fila"]==0)].size/k)
print("Tiempo promedio en el sistema:",simu2['tiempo en el sistema'].mean())

Clientes promedio en la fila: 1.13489
Probabilidad de cero clientes en la fila: 0.47033
Tiempo promedio en el sistema: 17.097155
